<a href="https://colab.research.google.com/github/DavidAde6/DavidAde6/blob/main/Sentinel-5P-NO2-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SETUP**

In [2]:
# IMPORTS
import ee
from datetime import datetime
import geemap.core as geemap
from IPython.display import display
import pandas as pd

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-dadeniyi54')
collection = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [21]:
region = ee.Geometry.BBox(-76.5, 44.9, -75, 45.7)
filtered_collection = collection.filterDate('2021-01-01', '2023-12-31').filterBounds(region)

In [22]:
def extract_data(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.BBox(-76.5, 44.9, -75, 45.7),
        scale=7000  # Sentinel-5P resolution
    )
    return ee.Feature(None, stats).set('date', image.date().format())

In [23]:
features = filtered_collection.map(extract_data).getInfo()
data = pd.DataFrame([feature['properties'] for feature in features['features']])
clean_data = data.dropna()

,NO2_column_number_density,NO2_slant_column_number_density,absorbing_aerosol_index,cloud_fraction,date,sensor_altitude,sensor_azimuth_angle,sensor_zenith_angle,solar_azimuth_angle,solar_zenith_angle,stratospheric_NO2_column_number_density,tropopause_pressure,tropospheric_NO2_column_number_density
0,0.000015,0.000102,-2.420222,0.595377,2021-01-01T17:49:59,833214.337321,71.977530,3.370646,-168.246057,69.326835,0.000028,16706.328957,NaN
1,NaN,NaN,NaN,NaN,2021-01-01T17:54:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,2021-01-02T15:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000045,0.000135,-1.490492,0.554145,2021-01-02T17:29:59,833404.585913,68.647205,29.867273,-173.036760,68.407386,0.000029,22966.120930,0.000020
4,0.000044,0.000133,-1.470539,0.546708,2021-01-02T17:34:59,833416.590356,68.589436,30.051076,-173.095480,68.445373,0.000029,22883.531210,0.000021
5,NaN,NaN,NaN,NaN,2021-01-03T17:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,2021-01-04T16:54:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000100,0.000160,-1.292608,0.894217,2021-01-04T18:34:39,833010.309793,-94.700515,46.439965,-158.832095,70.831057,0.000028,22102.448946,NaN
8,NaN,NaN,NaN,NaN,2021-01-04T18:39:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,2021-01-05T16:34:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# WE HAVE DATA, NOW TO START WORKING ON DESIGNING THE LSTM Model